# Batch rename

In [ ]:
import os

In [ ]:
def rename_images(folder_path, prefix="image"):
    # Check if the provided path exists and is a directory
    if not os.path.isdir(folder_path):
        print("Provided folder path does not exist or is not a directory.")
        return

    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter for image files (jpg, png, jpeg, etc.)
    image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

    # Loop through each image and rename it
    for index, filename in enumerate(image_files):
        # Set new name with prefix and index
        new_name = f"{prefix}_{filename}"

        # Full paths for renaming
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_name)

        # Rename file
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_name}")

    print("All images have been renamed successfully.")

In [ ]:
parent = "/content/drive/MyDrive/Colab Notebooks/Imagenes"
folder = "/2,5m90"
path = parent+folder
print(path)

/content/drive/MyDrive/Colab Notebooks/Imagenes/Anegado


In [ ]:
# Apply
rename_images(path, "AN")

DJI_0418.JPG
Renamed: DJI_0418.JPG -> AN_DJI_0418.JPG
DJI_0417.JPG
Renamed: DJI_0417.JPG -> AN_DJI_0417.JPG
DJI_0419.JPG
Renamed: DJI_0419.JPG -> AN_DJI_0419.JPG
All images have been renamed successfully.
